In [1]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: <object object at 0x7fa67755a500>
2.0.0+cu117


'%.4f'

In [2]:
if "train_x" not in globals():
    x, y = lib.datasets.get_mnist_train()
    train_y = y % 2
    train_x = torch.reshape(x, (len(x), -1))
    x, y = lib.datasets.get_mnist_test()
    test_y = y % 2
    test_x = torch.reshape(x, (len(x), -1))

b = train_x[:, 0].numpy()



In [40]:
lib.utils.set_seed()
# network = lib.nets.PYRAMID()
#network = lib.nets.create_livenet_odd_2()
# train_x, train_y = lib.datasets.get_odd()
# network = lib.livenet.LiveNet(train_x.shape[1], None, torch.max(train_y) + 1)
network = lib.nets.PERCEPTRON(train_x.shape[1], torch.max(train_y) + 1, l1=0.01)
criterion = lib.nets.criterion_n

def create_optimizer(net: nn.Module):
    if net.__class__.__name__ == "LiveNet":
        print("LiveNet")
        net: lib.livenet.LiveNet
        optimizer = lib.livenet.LiveNetOptimizer(net, decay=0.0)
        # optimizer = torch.optim.Adam(net.parameters())
    else:
        print("Torch")
        optimizer = torch.optim.Adam(net.parameters(), betas=(0.0, 0.95))
        # optimizer = torch.optim.Adam(net.parameters())
    return optimizer

optimizer = create_optimizer(network)


Torch


In [41]:
params = [p for p in network.named_parameters()]
logits = network.forward(train_x)
for name, p in params:
    print(f"{name} {p.detach().numpy()}")
pred = network.forward(train_x)
loss = criterion(pred, train_y)
print(loss)
pred.detach().numpy()
# torch.onnx.export(network, train_x, "/home/spometun/model.onnx", verbose=False)




linear1.weight [[-0.0003  0.0192 -0.0294 ...  0.0219  0.0037  0.0021]
 [-0.0198 -0.015  -0.0104 ... -0.0203 -0.006  -0.0299]]
linear1.bias [-0.0201  0.0149]
tensor(1.0976, grad_fn=<DivBackward0>)


array([[-0.0188,  0.0949],
       [-0.0753,  0.1432],
       [-0.0859,  1.0602],
       ...,
       [ 0.6132,  0.5901],
       [ 0.1025,  0.7495],
       [ 0.1927,  0.4021]], dtype=float32)

In [42]:
# network.context.decay = 0.00
params_history = []


In [45]:
optimizer.param_groups[0]["lr"] = 0.001
# network.alpha_l1 = 0.1
batch_size = 1000

n_epochs = 8
for i in range(n_epochs):
    e_counter = 0
    for start, end in lib.gen_utils.index_batcher(batch_size, len(train_x)):
        params = [(p[0], p[1].detach().clone().numpy()) for p in network.named_parameters()]
        params_history.append(params)
        x = train_x[start:end]
        y = train_y[start:end]
        pred = network.forward(x)
        loss = criterion(pred, y)
        reg_loss = network.internal_loss()
        all_loss = loss + reg_loss
        optimizer.zero_grad()
        all_loss.backward()
        optimizer.step()
        if e_counter == 0:
            LOG(f"{all_loss.detach().numpy():.3f} = {loss.detach().numpy():.3f}+{reg_loss.detach().numpy():.3f}")
        e_counter += 1



Iˈ0.000 0.562 = 0.450+0.112 
Iˈ0.072 0.561 = 0.448+0.113 
Iˈ0.132 0.563 = 0.451+0.113 
Iˈ0.192 0.560 = 0.447+0.113 
Iˈ0.259 0.562 = 0.449+0.113 
Iˈ0.337 0.559 = 0.446+0.113 
Iˈ0.397 0.563 = 0.449+0.113 
Iˈ0.457 0.559 = 0.446+0.113 


In [46]:

params_orig = [p.detach().clone().numpy() for p in network.parameters()]


In [47]:
# params = copy.deepcopy(params_orig)
#lib.utils.add_noise_to_params(params, 1, .0)
# lib.utils.set_parameters(network, params)


In [48]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.1312

In [49]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_picker0(param):
    return param[0][1][0][0].item()

def param_picker1(param):
    return param[0][1][1][0].item()
    # return param[1][1][0].item()
    # return np.max(np.abs(param[0].numpy()))

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(params_history, param_picker0)
plt.plot(values)
values = get_param_values(params_history, param_picker1)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(network.parameters())
# accum.add_value(params_history[-1])
# accum.plot()




Using matplotlib backend: TkAgg


In [39]:
def form_balanced(x, y, n):
    assert n % 2 == 0
    n //= 2
    y = y.numpy()
    x = x.numpy()
    inds = np.argsort(y.squeeze(1), axis=0)
    y = y[inds]
    x = x[inds]
    i = 0
    batches_x = []
    batches_y = []
    while True:
        a = y[i:i + n]
        b = y[len(y) - i - n: len(y) - i]
        if not (a==0).all() or not (b==1).all():
            break
        batches_x += [x[i:i+n], x[len(y) - i - n: len(y) - i]]
        batches_y += [a, b]
        i += n

    x = np.vstack(batches_x)
    y = np.vstack(batches_y)
    return torch.tensor(x), torch.tensor(y)

train_x, train_y = form_balanced(train_x, train_y, 1000)


In [ ]:
# list(network.parameters())[0].data = torch.Tensor([[10.], [-10]])
# list(network.parameters())[1].data = torch.Tensor([-5., 15])
np.set_printoptions(precision=2)
params = list(network.parameters())
par = []
for p in params:
    par.append(p.detach())
    print (par[-1])

In [ ]:
o1 = train_x @ par[1].T + par[2]
o1

In [ ]:
a1 = torch.sigmoid(o1)
a1

In [ ]:
o2 = a1[:, 0] + a1[:, 1] - 1
mseloss(o2, train_y)

In [ ]:
o2 @ par[2].T

In [ ]:
network.forward(train_x)

In [ ]:
1345639 * 5145

In [ ]:
import pickle
import numpy as np

class A:
    def __init__(self):
        self.v = 1
    def __del__(self):
        print(f"del {self.v}")

a = A()
b = A()
b.v = 2
l = [b]
print("start")
del a
print("after del")
l[0].v
a.v
